In [1]:
# Import required libraries
import numpy as np
import pandas as pd

import openmeteo_requests
import requests_cache
from retry_requests import retry

import matplotlib.pyplot as plt
import seaborn as sns

## Use command "pip freeze > requirements.txt" to save the dependencies in requirements.txt

### Open-Meteo Weather API - Doku
https://github.com/open-meteo/open-meteo?tab=readme-ov-file

Historical Weather: https://open-meteo.com/en/docs/historical-weather-api

Weather Forecast:   https://open-meteo.com/en/docs

### OpenStreetMap - Doku
https://www.openstreetmap.org/#map=8/46.826/8.613

### Task Overview

Step 1: Get daily and hourly historical Weather data for 5 specified coordinates

In [33]:
# Create a pandas dataframe with coordinates of locations to process
# Locations and coordinates found in openStreetMap
locations_coords = pd.DataFrame({
    "location" : ["Schönengrund, Hinterland, Appenzell Ausserrhoden, 9105, Switzerland", 
              "Einsiedeln, Schwyz, 8840, Switzerland",
              "Wolfenschiessen, Nidwalden, Switzerland",
              "Pontresina, Maloja, Grisons, 7504, Switzerland",
              "Ruvigliana, Circolo di Lugano est, Lugano, Distretto di Lugano, Ticino, 6977, Switzerland"
              ], 
    "lat" : [47.31706, 47.1071, 46.9062, 46.4774, 46.00865], 
    "lon" : [9.24482, 8.8114, 8.4282, 9.9042, 8.98767]})
print(locations_coords)


                                            location       lat      lon
0  Schönengrund, Hinterland, Appenzell Ausserrhod...  47.31706  9.24482
1              Einsiedeln, Schwyz, 8840, Switzerland  47.10710  8.81140
2            Wolfenschiessen, Nidwalden, Switzerland  46.90620  8.42820
3     Pontresina, Maloja, Grisons, 7504, Switzerland  46.47740  9.90420
4  Ruvigliana, Circolo di Lugano est, Lugano, Dis...  46.00865  8.98767


In [38]:
print(type(list(locations_coords.loc[:, "location"])))
print(list(locations_coords.loc[:, "location"]))

<class 'list'>
['Schönengrund, Hinterland, Appenzell Ausserrhoden, 9105, Switzerland', 'Einsiedeln, Schwyz, 8840, Switzerland', 'Wolfenschiessen, Nidwalden, Switzerland', 'Pontresina, Maloja, Grisons, 7504, Switzerland', 'Ruvigliana, Circolo di Lugano est, Lugano, Distretto di Lugano, Ticino, 6977, Switzerland']


In [53]:
# Define a start and end date of data to fetch from Open-Meteo API
start_date = "2024-01-01"
end_date = "2024-09-09"

In [54]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)     # stores data for 1 hour in the cache
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)         # retries 5 times with increasing delay of 20%
openmeteo = openmeteo_requests.Client(session = retry_session)                  # Using the retry_session to make a request to Open-Meteo API

In [55]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": list(locations_coords["lat"]),
	"longitude": list(locations_coords["lon"]),
	"start_date": start_date,
	"end_date": end_date,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m", "uv_index", "uv_index_clear_sky", "is_day", "sunshine_duration"],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", "precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max", "wind_speed_10m_max", "wind_gusts_10m_max"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)
print(f"Number of resposes: {len(responses)}")

Number of resposes: 5


In [56]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[4]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 46.02000045776367°N 9.0°E
Elevation 926.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s


In [57]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_showers = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
hourly_visibility = hourly.Variables(15).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(18).ValuesAsNumpy()
hourly_uv_index = hourly.Variables(19).ValuesAsNumpy()
hourly_uv_index_clear_sky = hourly.Variables(20).ValuesAsNumpy()
hourly_is_day = hourly.Variables(21).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(22).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["lat"] = response.Latitude()
hourly_data["lon"] = response.Longitude()


hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["visibility"] = hourly_visibility
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["uv_index"] = hourly_uv_index
hourly_data["uv_index_clear_sky"] = hourly_uv_index_clear_sky
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe.shape)
print(hourly_dataframe)

(6072, 26)
                          date    lat  lon  temperature_2m  \
0    2023-12-31 22:00:00+00:00  46.02  9.0        1.977000   
1    2023-12-31 23:00:00+00:00  46.02  9.0        1.577000   
2    2024-01-01 00:00:00+00:00  46.02  9.0        1.842500   
3    2024-01-01 01:00:00+00:00  46.02  9.0        2.542500   
4    2024-01-01 02:00:00+00:00  46.02  9.0        2.542500   
...                        ...    ...  ...             ...   
6067 2024-09-09 17:00:00+00:00  46.02  9.0       18.592501   
6068 2024-09-09 18:00:00+00:00  46.02  9.0       17.942501   
6069 2024-09-09 19:00:00+00:00  46.02  9.0       17.492500   
6070 2024-09-09 20:00:00+00:00  46.02  9.0       17.642500   
6071 2024-09-09 21:00:00+00:00  46.02  9.0       17.142500   

      relative_humidity_2m  precipitation_probability  precipitation  rain  \
0                     99.0                        NaN            0.0   0.0   
1                     97.0                        NaN            0.0   0.0   
2         

In [58]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
daily_sunrise = daily.Variables(5).ValuesAsNumpy()
daily_sunset = daily.Variables(6).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(7).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(8).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(9).ValuesAsNumpy()
daily_uv_index_clear_sky_max = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_showers_sum = daily.Variables(13).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(14).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(15).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(17).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(18).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["lat"] = response.Latitude()
daily_data["lon"] = response.Longitude()

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["uv_index_clear_sky_max"] = daily_uv_index_clear_sky_max
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["showers_sum"] = daily_showers_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

                         date    lat  lon  weather_code  temperature_2m_max  \
0   2023-12-31 22:00:00+00:00  46.02  9.0          45.0            5.842500   
1   2024-01-01 22:00:00+00:00  46.02  9.0           3.0            4.542500   
2   2024-01-02 22:00:00+00:00  46.02  9.0           3.0            6.192500   
3   2024-01-03 22:00:00+00:00  46.02  9.0           3.0            8.542500   
4   2024-01-04 22:00:00+00:00  46.02  9.0          63.0            5.442500   
..                        ...    ...  ...           ...                 ...   
248 2024-09-04 22:00:00+00:00  46.02  9.0          96.0           17.792500   
249 2024-09-05 22:00:00+00:00  46.02  9.0          95.0           18.292500   
250 2024-09-06 22:00:00+00:00  46.02  9.0           3.0           20.692501   
251 2024-09-07 22:00:00+00:00  46.02  9.0          96.0           18.042500   
252 2024-09-08 22:00:00+00:00  46.02  9.0          95.0           21.092501   

     temperature_2m_min  apparent_temperature_max  

In [60]:
# Save dataframes as csv files
hourly_dataframe.to_csv("historic_weather_hourly.csv")
daily_dataframe.to_csv("historic_weather_daily.csv")